In [ ]:
!pip install torchio
%pip install --quiet matplotlib
%pip install --quiet torchvision
!apt -qq install tree

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.8/172.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.5/237.5 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: rich
    Found existing installation: rich 13.3.4
    Uninstalling rich-13.3.4:
      Successfully uninstalled rich-13.3.4
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 43.0 kB of archives.
After this operation, 115 kB of additional disk space will be used.
Selecting previously unselected package tree.
(Reading database ... 122518 files and directories currently installed.)
Preparing to unpack .../tree_1.8.0-1_amd64.deb ...
Unpacking tree (1.8.0-1) ...
Setting up tre

In [ ]:
import os
import enum
import tqdm
import numpy as np
import time
import torch.nn.functional as F
import torch
import torchio as tio
from torch.utils.data import DataLoader
import multiprocessing
import torch.backends.cudnn as cudnn
from torch.utils.data import ConcatDataset
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier



num_workers = multiprocessing.cpu_count()

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)


device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'

Sun May  7 21:00:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /gdrive

Mounted at /content/drive
[Errno 2] No such file or directory: '/gdrive'
/content


In [ ]:
import sys
sys.path.insert(0,"/content/drive/MyDrive/IMA205/ima205-challenge-2023")

In [ ]:
data_path = "/content/drive/MyDrive/IMA205/ima205-challenge-2023"


for file in sorted(os.listdir(data_path)):
  print(file)

.DS_Store
Classif_model.pt
SampleSubmission.csv
Segmentation_model_.pt
Test
Train
best_models
metaDataTest.csv
metaDataTrain.csv
new_segmentation_models
results
segmentation_model_ED.pt
segmentation_model_ES.pt


In [ ]:
import pandas as pd
df_meta_test = pd.read_csv(os.path.join(data_path, 'metaDataTest.csv'))
df_meta_train = pd.read_csv(os.path.join(data_path, 'metaDataTrain.csv'))


In [ ]:
training_transform = tio.Compose([
    #tio.ToCanonical(),
    tio.Resample((1.25, 1.25, 10)),
    #tio.RandomMotion(p=0.2),
    tio.CropOrPad((200, 200, 10)),
    tio.ZNormalization(masking_method=tio.ZNormalization.mean),
    #tio.OneHot(),
])

validation_transform = tio.Compose([
    #tio.ToCanonical(),
    tio.Resample((1.25, 1.25, 10)),
    tio.CropOrPad((200, 200, 10)),
    tio.ZNormalization(masking_method=tio.ZNormalization.mean),
    #tio.OneHot(),
])


In [ ]:
# create a dataset of first type of mri and the second one
from random import shuffle

def counter(a):
  if a >= 100:
    return str(a)
  elif a < 10:
    return '00'+str(a)
  else:
    return '0'+str(a)


#subjects_ED = []
#subjects_ES = []

subjects_mixed = []


permute = np.random.permutation(100)+1


for i in permute:
  current_dir = os.path.join(data_path, 'Train', counter(i))
  subject_ED = tio.Subject(
        mri=tio.ScalarImage(os.path.join(current_dir, counter(i)+'_ED.nii')),
        segment=tio.LabelMap(os.path.join(current_dir, counter(i)+'_ED_seg.nii')),
        height=df_meta_train.iloc[i-1]['Height'],
        weight=df_meta_train.iloc[i-1]['Weight'],
        category=df_meta_train.iloc[i-1]['Category']

    )

  subject_ES = tio.Subject(
        mri=tio.ScalarImage(os.path.join(current_dir, counter(i)+'_ES.nii')),
        segment=tio.LabelMap(os.path.join(current_dir, counter(i)+'_ES_seg.nii')),
        height=df_meta_train.iloc[i-1]['Height'],
        weight=df_meta_train.iloc[i-1]['Weight'],
        category=df_meta_train.iloc[i-1]['Category']
    )


  subjects_mixed.append(subject_ED)
  subjects_mixed.append(subject_ES)


shuffle(subjects_mixed)



dataset_train = tio.SubjectsDataset(subjects_mixed, transform=training_transform)

'''
lengths = [int(len(dataset_ED_train)*0.8), int(len(dataset_ED_train)*0.2)]
train_ED, valid_ED = torch.utils.data.random_split(dataset_ED_train, lengths)
lengths = [int(len(dataset_ES_train)*0.8), int(len(dataset_ES_train)*0.2)]
train_ES, valid_ES = torch.utils.data.random_split(dataset_ES_train, lengths)
'''
print('Dataset size:', len(dataset_train), 'subjects')

Dataset size: 200 subjects


In [ ]:
# create a dataset of first type of mri and the second one

subjects_ED = []
subjects_ES = []
for i in range(101,151):
  current_dir = os.path.join(data_path, 'Test', counter(i))
  subject_ED = tio.Subject(
        mri=tio.ScalarImage(os.path.join(current_dir, counter(i)+'_ED.nii')),
        segment=tio.LabelMap(os.path.join(current_dir, counter(i)+'_ED_seg.nii')),
        height=df_meta_test.iloc[i-100-1]['Height'],
        weight=df_meta_test.iloc[i-100-1]['Weight'],
    )
  subject_ES = tio.Subject(
        mri=tio.ScalarImage(os.path.join(current_dir, counter(i)+'_ES.nii')),
        segment=tio.LabelMap(os.path.join(current_dir, counter(i)+'_ES_seg.nii')),
        height=df_meta_test.iloc[i-100-1]['Height'],
        weight=df_meta_test.iloc[i-100-1]['Weight'],
    )

  subjects_ED.append(subject_ED)
  subjects_ES.append(subject_ES)



dataset_ED_test = tio.SubjectsDataset(subjects_ED, transform=validation_transform)
dataset_ES_test = tio.SubjectsDataset(subjects_ES, transform=validation_transform)


print('Dataset size:', len(dataset_ED_test), 'subjects')

Dataset size: 50 subjects


In [ ]:
training_batch_size = 4
validation_batch_size = 4


training_loader = torch.utils.data.DataLoader(
    dataset_train,
    batch_size=training_batch_size,
    shuffle=True,
    num_workers=num_workers,
)


test_ED_loader = torch.utils.data.DataLoader(
    dataset_ED_test,
    batch_size=validation_batch_size,
    num_workers=num_workers)

test_ES_loader = torch.utils.data.DataLoader(
    dataset_ES_test,
    batch_size=validation_batch_size,
    num_workers=num_workers)


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'

import torch
import torch.nn as nn

# Define the network architecture
class DilatedConvNet(nn.Module):
    def __init__(self):
        super(DilatedConvNet, self).__init__()


        self.encoder_1 = nn.Sequential(nn.Conv3d(1, 4, kernel_size=(5,5,1), padding=(2,2,0), dilation=(1,1,1)),
                                     nn.BatchNorm3d(4),
                                     nn.ReLU(inplace=True),
                                     nn.MaxPool3d((2, 2, 1), stride=(2, 2, 1))) # 100

        self.encoder_2 = nn.Sequential(nn.Conv3d(4, 8, kernel_size=(5,5,1), padding=(2,2,0), dilation=(1,1,1)),
                                     nn.BatchNorm3d(8),
                                     nn.ReLU(inplace=True),
                                     nn.MaxPool3d((2, 2, 1), stride=(2, 2, 1))) # 50

        self.encoder_3 = nn.Sequential(nn.Conv3d(8, 16, kernel_size=(5,5,1), padding=(2,2,0), dilation=(1,1,1)),
                                     nn.BatchNorm3d(16),
                                     nn.ReLU(inplace=True),
                                     nn.MaxPool3d((2, 2, 1), stride=(2, 2, 1))) # 25

        self.encoder_4 = nn.Sequential(nn.Conv3d(16, 32, kernel_size=(5,5,1), padding=(2,2,0), dilation=(1,1,1)),
                                     nn.BatchNorm3d(32),
                                     nn.ReLU(inplace=True),
                                     nn.MaxPool3d((2, 2, 1), stride=(2, 2, 1))) # 12

        self.encoder_5 = nn.Sequential(nn.Conv3d(32, 64, kernel_size=(5,5,1), padding=(2,2,0), dilation=(1,1,1)),
                                     nn.BatchNorm3d(64),
                                     nn.ReLU(inplace=True),
                                     nn.MaxPool3d((2, 2, 1), stride=(2, 2, 1))) # (batch_size, 64, 6, 6)


        # should use ConvTranspose3d to inverse the maxpooling
        # (D_in−1)×stride[0]−2×padding[0]+dilation[0]×(kernel_size[0]−1)+1

        # 6 to 12 => 12 = 5*2 - 2*1 + 1*3 + 1
        self.decoder_5 = nn.Sequential(nn.ConvTranspose3d(64, 32, (4, 4, 1), stride=(2, 2, 1), padding=(1, 1, 0)), # (batch_size,8,50,50,5)
                                     nn.BatchNorm3d(32),
                                     nn.ReLU(inplace=True)) # (batch_size, 32, 12, 12)
        # we should combine the output of decoder_5 with the output of encoder_4 and pass it to
        # decoder_4

        # 12 to 25 => 25 = 11*2 - 2*1 + 1*4 + 1
        self.decoder_4 = nn.Sequential(nn.ConvTranspose3d(64, 32, (5, 5, 1), stride=(2, 2, 1), padding=(1, 1, 0)), # (batch_size,8,50,50,5)
                                     nn.BatchNorm3d(32),
                                     nn.ReLU(inplace=True),
                                     nn.Conv3d(32, 16, kernel_size=(1,1,1)),
                                     nn.ReLU(inplace=True))


        # 25 to 50 => 50 = 24*2 - 2*1 + 1*3 + 1
        self.decoder_3 = nn.Sequential(nn.ConvTranspose3d(32, 16, (4, 4, 1), stride=(2, 2, 1), padding=(1, 1, 0)), # (batch_size,8,50,50,5)
                                     nn.BatchNorm3d(16),
                                     nn.ReLU(inplace=True),
                                     nn.Conv3d(16, 8, kernel_size=(1,1,1)),
                                     nn.ReLU(inplace=True))


        # 50 to 100 => 50 = 24*2 - 2*1 + 1*3 + 1
        self.decoder_2 = nn.Sequential(nn.ConvTranspose3d(16, 8, (4, 4, 1), stride=(2, 2, 1), padding=(1, 1, 0)), # (batch_size,8,50,50,5)
                                     nn.ReLU(inplace=True),
                                     nn.Conv3d(8, 4, kernel_size=(1,1,1)),
                                     nn.ReLU(inplace=True))


        # 100 to 200 => 200 = 99*2 - 2*1 + 1*3 + 1
        self.decoder_1 = nn.Sequential(nn.ConvTranspose3d(8, 4, (4, 4, 1), stride=(2, 2, 1), padding=(1, 1, 0)), # (batch_size,8,50,50,5)
                                     nn.ReLU(inplace=True),
                                     nn.Conv3d(4, 1, kernel_size=(1,1,1)),
                                     #nn.ReLU(inplace=True)
                                     )



    def forward(self, input, mode='train'):
        # Input shape: (batch_size, 281, 281, 10)
        # we convert it to (batch_size, 3, 281, 281, 10)

        seg_myo = (input['segment'][tio.DATA][:,0,:]==2).int().unsqueeze(1).to(device).float()


        features = []
        x = seg_myo
        features.append(x)
        x = self.encoder_1(x)
        features.append(x)
        x = self.encoder_2(x)
        features.append(x)
        x = self.encoder_3(x)
        features.append(x)
        x = self.encoder_4(x)
        features.append(x)
        x = self.encoder_5(x)


        x = self.decoder_5(x)
        #print(x.size())
        x = torch.cat((x,features[-1]), dim=1)
        #print(x.size())
        x = self.decoder_4(x)
        x = torch.cat((x,features[-2]), dim=1)
        x = self.decoder_3(x)
        x = torch.cat((x,features[-3]), dim=1)
        x = self.decoder_2(x)
        x = torch.cat((x,features[-4]), dim=1)
        x = self.decoder_1(x)

        output = x

        return output



# train a model for 300 epochs, where each epoch is 100 batches, each batch contains 4 samples
# 300*100 = 30000
# so, in each epoch, we go through the whole dataset (in average, but we impose this average in the code)
# 4 times => n_steps_in_epoch
def train(model, data_loader, epochs=300, n_sets_in_epoch=4, lr=0.005):

    optimizer = optim.Adam(model.parameters(), lr=lr)
    schedualr = optim.lr_scheduler.StepLR(optimizer, 100, gamma=0.98)
    # scheduler = CyclicLR(optimizer, base_lr=0.2)

    # Define the loss function and the device
    criterion = nn.BCEWithLogitsLoss()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Move the model and the loss function to the device
    model = model.to(device)
    criterion = criterion.to(device)


    model.eval() if optimizer is None else model.train()
    train_loss_list = []

    for epoch in range(1, epochs + 1):
        # Train the model
        model.train()
        train_loss = 0.0

        for n_set in range(n_sets_in_epoch):

            for i, input in enumerate(data_loader):

              target = (input['segment'][tio.DATA]==3).int().to(device).float()
              optimizer.zero_grad()
              output = model(input)
              output_ = output.cpu()
              output_ = output_[0,0,:,:,4]
              print(output_.size())

              plt.imshow(output_.detach().numpy())
              plt.show()

              loss = criterion(output, target)
              print('loss in current batch ', loss.item())

              loss.backward()
              optimizer.step()
              schedualr.step()

              train_loss += loss.item()



        # Calculate the epoch loss
        train_loss /= len(data_loader)
        print('loss in the epoch ', train_loss)
        train_loss_list.append(train_loss)


    # Return the snapshot ensemble lists, the best model and its corresponding validation loss
    return train_loss_list

In [ ]:
model = DilatedConvNet()
train(model, training_loader)
torch.save(model.state_dict(), os.path.join(data_path, 'new_segmentation_models', 'Segmentation_model_.pt'))